In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda


training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(
    training_data, batch_size=64
)
test_dataloader = DataLoader(
    test_data, batch_size=64
)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
loss_fn = nn.CrossEntropyLoss()

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epochs {t+1}\n ----------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('Done!')

Epochs 1
 ----------------------
loss: 2.313511 [    0/60000]
loss: 2.297167 [ 6400/60000]
loss: 2.281606 [12800/60000]
loss: 2.268901 [19200/60000]
loss: 2.251368 [25600/60000]
loss: 2.232074 [32000/60000]
loss: 2.237199 [38400/60000]
loss: 2.212439 [44800/60000]
loss: 2.197407 [51200/60000]
loss: 2.173473 [57600/60000]
Test Error: 
 Accuracy: 48.0%, Avg loss: 2.167256 

Epochs 2
 ----------------------
loss: 2.181800 [    0/60000]
loss: 2.168237 [ 6400/60000]
loss: 2.119170 [12800/60000]
loss: 2.125786 [19200/60000]
loss: 2.079686 [25600/60000]
loss: 2.030833 [32000/60000]
loss: 2.052356 [38400/60000]
loss: 1.986548 [44800/60000]
loss: 1.972642 [51200/60000]
loss: 1.911683 [57600/60000]
Test Error: 
 Accuracy: 58.8%, Avg loss: 1.909178 

Epochs 3
 ----------------------
loss: 1.946215 [    0/60000]
loss: 1.912132 [ 6400/60000]
loss: 1.806074 [12800/60000]
loss: 1.833210 [19200/60000]
loss: 1.730707 [25600/60000]
loss: 1.686114 [32000/60000]
loss: 1.704404 [38400/60000]
loss: 1.616711